In [ ]:
# default_exp simulators.projectq

# ProjectQ Statevector Simulator

> Wrapper for ProjectQ Python library https://github.com/ProjectQ-Framework/ProjectQ

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from qsam.simulators.mixins import CircuitRunnerMixin

from projectq import MainEngine
import projectq.ops as ops

In [ ]:
#export
class MeasureResult:
    
    def __init__(self, value: bool):
        self.value = bool(value)
        
    def __bool__(self):
        return self.value

In [ ]:
#export
class StateVectorSimulator(CircuitRunnerMixin):
    
    GATES = {
        "1qb": {"I", "X", "Y", "Z", "H", "T", "Q", "Qd", "S", "Sd", "R", "Rd", "Rx", "Ry", "Rz"},
        "2qb": {"CNOT", "MSd"},
        "meas": {"measure"},
        "init": {"init"}
    }
        
    def __init__(self, num_qubits):
        self._n = num_qubits
        self.eng = MainEngine()
        self.qureg = self.eng.allocate_qureg(num_qubits)
        self.qubits = {i:qb for i,qb in enumerate(self.qureg)}
        
    def init(self, qubit: int) -> None:
        outcome = self.measure(qubit)
        if outcome.value == 1:
            self.X(qubit)
    
    def measure(self, qubit: int) -> MeasureResult:
        q = self.qubits[qubit]
        self.eng.flush()
        ops.Measure | q
        self.eng.flush()
        return MeasureResult(value=int(q)) 
    
    def expectation(self, qubit: int) -> float:
        self.eng.flush()
        exp_op = ops.QubitOperator(f'Z{qubit}')
        exp = self.eng.backend.get_expectation_value(exp_op, self.qureg)
        ops.All(ops.Measure) | self.qureg # To avoid error message of deallocating qubits in a superposition
        return exp
        
    def I(self, qubit: int) -> None:
        pass
    
    def X(self, qubit: int) -> None:
        ops.X | self.qubits[qubit]
        
    def Y(self, qubit: int) -> None:
        ops.Y | self.qubits[qubit]
        
    def Z(self, qubit: int) -> None:
        ops.Z | self.qubits[qubit]
        
    def H(self, qubit: int) -> None:
        ops.H | self.qubits[qubit]
        
    def CNOT(self, control: int, target: int) -> None:
        ops.CNOT | (self.qubits[control], self.qubits[target])
        
    def T(self, qubit: int) -> None:
        ops.T | self.qubits[qubit]
        
    def Td(self, qubit: int) -> None:
        ops.Tdagger | self.qubits[qubit]
        
    def S(self, qubit: int) -> None:
        ops.S | self.qubits[qubit]
        
    def Sd(self, qubit: int) -> None:
        ops.Sdagger | self.qubits[qubit]
        
    def Q(self, qubit: int) -> None:
        ops.SqrtX | self.qubits[qubit]
        
    def Qd(self, qubit: int) -> None:
        Qd = ops.get_inverse(ops.SqrtX)
        Qd | self.qubits[qubit]
    
    def Rx(self, qubit: int, angle: float) -> None:
        ops.Rx(angle) | self.qubits[qubit]
        
    def Ry(self, qubit: int, angle: float) -> None:
        ops.Ry(angle) | self.qubits[qubit]
   
    def Rz(self, qubit: int, angle: float) -> None:
        ops.Rz(angle) | self.qubits[qubit]
        
    def R(self, qubit: int) -> None:
        ops.R | self.qubits[qubit]
        
    def Rd(self, qubit: int) -> None:
        Rd = ops.get_inverse(ops.R)
        Rd | self.qubits[qubit]
        
    def MSd(self, qubitA: int, qubitB: int) -> None:
        self.Rd(qubitA)
        self.CNOT(qubitA, qubitB)
        self.R(qubitA)
        self.Qd(qubitA)
        self.Qd(qubitB)
        
    def __del__(self):
        self.eng.flush()
        ops.All(ops.Measure) | self.qureg # Required by ProjectQ
        
        try:
            self.eng.flush(deallocate_qubits=True)
        except KeyError:
            pass

Tests

In [ ]:
s = StateVectorSimulator(num_qubits=3)
s.Ry(0,1.0)
s.expectation(0)

0.5403023058681398